# 数据预处理与特征工程1

想象一下未来美好的一天，你学完了菜菜的课程，成为一个精通各种算法和调参调库的数据挖掘工程师了。某一天你从你的同事，一位药物研究人员那里，得到了一份病人临床表现的数据。药物研究人员用前四列数据预测一下最后一数据，还说他要出差几天，可能没办法和你一起研究数据了，希望出差回来以后，可以有个初步分析结果。于是你就看了看数据，看着很普通，预测连续型变量，好说，导随机森林回归器调出来，调参调呀调，MSE很小，跑了个还不错的结果。

![image.png](https://pic1.zhimg.com/80/v2-f523da29c40e71622a54eb7eab458bdc_720w.jpg)

几天后，你同事出差回来了，准备要一起开会了，会上你碰见了和你同事在同一个项目里工作的统计学家。他问起你的分析结果，你说你已经小有成效了，统计学家很吃惊，他说：“不错呀，这组数据问题太多，我都分析不出什么来。”

你心里可能咯噔一下，忐忑地回答说：“我没听说数据有什么问题呀。”

于是统计学家说：“诶？没人告诉你说，最后一列数据如果取个对数，结果会更好吗？”

你内心毫无波动：“没。”

统计学家：“诶那你肯定听说了第四列数据有点问题吧，这个特征的取值范围是1~10，0是表示缺失值的。而且他们输入数据的时候出错，很多10都被录入成0了，现在分不出来了。”

你：”......“

统计学家：”还有第二列和第三列数据基本是一样的，相关性太强了。“

你：”这个我发现了，不过这两个特征在预测中的重要性都不高，无论其他特征怎样出错，我这边结果里显示第一列的特征是最重要的，所以也无所谓啦。“

统计学家：“啥？第一列不就是编号吗？”

你：“不是吧。”

统计学家：“哦我想起来了！第一列就是编号，不过那个编号是我们根据第五列排序之后编上去的！这个第一列和第五列是由很强的联系，但是毫无意义啊！”

老血喷了一屏幕，数据挖掘工程师卒。

这个悲惨又可爱的故事来自《数据挖掘导论》，虽然这是故事里的状况十分极端，但我还是想把这段对话作为今天这章的开头，博大家一笑（虽然可能听完就泪流满面了）。在过去两周，我们已经讲了两个算法：决策树和随机森林，我们通过决策树带大家认识了sklearn，通过随机森林讲解了机器学习中调参的基本思想，现在可以说，只要上过前面两堂课的，人人都会调随机森林和决策树的分类器了，而我呢，也只需要跟着各大机器学习书籍的步伐，给大家一周一个算法带着讲解就是了。如果这样的话，结果可能就是，大家去工作了，遇到了一个不那么靠谱的同事，给了你一组有坑的数据，最后你就一屏幕老血吐过去，牺牲在数据行业的前线了。

**数据不给力，再高级的算法都没有用。**

## 数据挖掘的五大流程：

**1. 获取数据**

**2. 数据预处理**

数据预处理是从数据中检测，纠正或删除损坏，不准确或不适用于模型的记录的过程
可能面对的问题有：数据类型不同，比如有的是文字，有的是数字，有的含时间序列，有的连续，有的间断。也可能，数据的质量不行，有噪声，有异常，有缺失，数据出错，量纲不一，有重复，数据是偏态，数据量太大或太小<br>
数据预处理的目的：让数据适应模型，匹配模型的需求

**3. 特征工程：**

特征工程是将原始数据转换为更能代表预测模型的潜在问题的特征的过程，可以通过挑选最相关的特征，提取特征以及创造特征来实现。其中创造特征又经常以降维算法的方式实现。 可能面对的问题有：特征之间有相关性，特征和标签无关，特征太多或太小，或者干脆就无法表现出应有的数据现象或无法展示数据的真实面貌<br>
特征工程的目的：1) 降低计算成本，2) 提升模型上限

**4. 建模，测试模型并预测出结果**

**5. 上线，验证模型效果**

## 1、sklearn中的数据预处理和特征工程

sklearn中包含众多数据预处理和特征工程相关的模块，虽然刚接触sklearn时，大家都会为其中包含的各种算法的广度深度所震惊，但其实sklearn六大板块中有两块都是关于数据预处理和特征工程的，两个板块互相交互，为建模之前的全部工程打下基础。

![image.png](https://pic3.zhimg.com/80/v2-426ddeb54406fa491a494e8c88940f52_720w.jpg)

- 模块preprocessing：几乎包含数据预处理的所有内容
- 模块Impute：填补缺失值专用
- 模块feature_selection：包含特征选择的各种方法的实践
- 模块decomposition：包含降维算法

## 2、数据预处理 Preprocessing & Impute

### 2.1 数据无量纲化

在机器学习算法实践中，我们往往有着将不同规格的数据转换到同一规格，或不同分布的数据转换到某个特定分布的需求，这种需求统称为将数据“无量纲化”。譬如梯度和矩阵为核心的算法中，譬如逻辑回归，支持向量机，神经网络，无量纲化可以加快求解速度；而在距离类模型，譬如K近邻，K-Means聚类中，无量纲化可以帮我们提升模型精度，避免某一个取值范围特别大的特征对距离计算造成影响。（一个特例是决策树和树的集成算法们，对决策树我们不需要无量纲化，决策树可以把任意数据都处理得很好。）

数据的无量纲化可以是线性的，也可以是非线性的。线性的无量纲化包括中心化（Zero-centered或者Mean-subtraction）处理和缩放处理（Scale）。中心化的本质是让所有记录减去一个固定值，即让数据样本数据平移到某个位置。缩放的本质是通过除以一个固定值，将数据固定在某个范围之中，取对数也算是一种缩放处理。

- **preprocessing.MinMaxScaler**

当数据(x)按照最小值中心化后，再按极差（最大值 - 最小值）缩放，数据移动了最小值个单位，并且会被收敛到[0,1]之间，而这个过程，就叫做**数据归一化(Normalization，又称Min-Max Scaling)**。注意，Normalization是归一化，不是正则化，真正的正则化是regularization，不是数据预处理的一种手段。归一化之后的数据服从正态分布，公式如下：

![image.png](https://www.zhihu.com/equation?tex=x%5E%2A+%3D+%5Cfrac%7Bx+-+min%28x%29%7D%7Bmax%28x%29-min%28x%29%7D%5C%5C)

在sklearn当中，我们使用**preprocessing.MinMaxScaler**来实现这个功能。MinMaxScaler有一个重要参数，feature_range，控制我们希望把数据压缩到的范围，默认是[0,1]。

In [1]:
from sklearn.preprocessing import MinMaxScaler

data = [[-1, 2], [-0.5, 6], [0, 10], [1, 18]]

In [2]:
#不太熟悉numpy的小伙伴，能够判断data的结构吗？
#如果换成表是什么样子？
import pandas as pd
pd.DataFrame(data)

0   1
0 -1.0   2
1 -0.5   6
2  0.0  10
3  1.0  18

In [3]:
#实现归一化
scaler = MinMaxScaler()                             #实例化
scaler = scaler.fit(data)                           #fit，在这里本质是生成min(x)和max(x)
result = scaler.transform(data)                     #通过接口导出结果
result

array([[0.  , 0.  ],
       [0.25, 0.25],
       [0.5 , 0.5 ],
       [1.  , 1.  ]])

In [4]:
result_ = scaler.fit_transform(data)                #训练和导出结果一步达成

In [5]:
result_

array([[0.  , 0.  ],
       [0.25, 0.25],
       [0.5 , 0.5 ],
       [1.  , 1.  ]])

In [6]:
scaler.inverse_transform(result)                    #将归一化后的结果逆转

array([[-1. ,  2. ],
       [-0.5,  6. ],
       [ 0. , 10. ],
       [ 1. , 18. ]])

In [7]:
#使用MinMaxScaler的参数feature_range实现将数据归一化到[0,1]以外的范围中

data = [[-1, 2], [-0.5, 6], [0, 10], [1, 18]]
scaler = MinMaxScaler(feature_range=[5,10])         #依然实例化
result = scaler.fit_transform(data)                 #fit_transform一步导出结果
result

#当X中的特征数量非常多的时候，fit会报错并表示，数据量太大了我计算不了
#此时使用partial_fit作为训练接口
#scaler = scaler.partial_fit(data)

array([[ 5.  ,  5.  ],
       [ 6.25,  6.25],
       [ 7.5 ,  7.5 ],
       [10.  , 10.  ]])

**使用numpy来实现归一化：**

In [8]:
import numpy as np
X = np.array([[-1, 2], [-0.5, 6], [0, 10], [1, 18]])

In [9]:
X.min()

-1.0

In [10]:
X.min(axis=1)

array([-1. , -0.5,  0. ,  1. ])

In [11]:
X.min(axis=0)

array([-1.,  2.])

In [12]:
#归一化
X_nor = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_nor

array([[0.  , 0.  ],
       [0.25, 0.25],
       [0.5 , 0.5 ],
       [1.  , 1.  ]])

In [13]:
#逆转归一化
X_returned = X_nor * (X.max(axis=0) - X.min(axis=0)) + X.min(axis=0)
X_returned

array([[-1. ,  2. ],
       [-0.5,  6. ],
       [ 0. , 10. ],
       [ 1. , 18. ]])

- **preprocessing.StandardScaler**

当数据(x)按均值(μ)中心化后，再按标准差(σ)缩放，数据就会服从为均值为0，方差为1的正态分布（即标准正态分布），而这个过程，就叫做**数据标准化**(Standardization，又称Z-score normalization)，公式如下：

![image.png](https://www.zhihu.com/equation?tex=x%5E%2A+%3D+%5Cfrac%7Bx-%CE%BC%7D%5Csigma%5C%5C)

In [14]:
from sklearn.preprocessing import StandardScaler
data = [[-1, 2], [-0.5, 6], [0, 10], [1, 18]]

In [15]:
scaler = StandardScaler()  #实例化
scaler.fit(data)      #fit，本质是生成均值和方差

StandardScaler()

In [17]:
scaler.mean_  #查看均值的属性mean_

array([-0.125,  9.   ])

In [16]:
scaler.var_                                         #查看方差的属性var_

array([ 0.546875, 35.      ])

In [18]:
x_std = scaler.transform(data)                      #通过接口导出结果

In [19]:
x_std

array([[-1.18321596, -1.18321596],
       [-0.50709255, -0.50709255],
       [ 0.16903085,  0.16903085],
       [ 1.52127766,  1.52127766]])

In [20]:
x_std.mean()    #导出的结果是一个数组，用mean()查看均值

0.0

In [21]:
x_std.std()    #用std()查看方差

1.0

In [22]:
scaler.fit_transform(data)     #使用fit_transform(data)一步达成结果

array([[-1.18321596, -1.18321596],
       [-0.50709255, -0.50709255],
       [ 0.16903085,  0.16903085],
       [ 1.52127766,  1.52127766]])

In [23]:
scaler.inverse_transform(x_std)    #使用inverse_transform逆转标准化

array([[-1. ,  2. ],
       [-0.5,  6. ],
       [ 0. , 10. ],
       [ 1. , 18. ]])

对于StandardScaler和MinMaxScaler来说，空值NaN会被当做是缺失值，在fit的时候忽略，在transform的时候保持缺失NaN的状态显示。并且，尽管去量纲化过程不是具体的算法，但在fit接口中，依然只允许导入至少二维数组，一维数组导入会报错。通常来说，我们输入的X会是我们的特征矩阵，现实案例中特征矩阵不太可能是一维所以不会存在这个问题。

- **StandardScaler和MinMaxScaler选哪个？**

看情况。大多数机器学习算法中，会选择StandardScaler来进行特征缩放，因为MinMaxScaler对异常值非常敏感。在PCA，聚类，逻辑回归，支持向量机，神经网络这些算法中，StandardScaler往往是最好的选择。

MinMaxScaler在不涉及距离度量、梯度、协方差计算以及数据需要被压缩到特定区间时使用广泛，比如数字图像处理中量化像素强度时，都会使用MinMaxScaler将数据压缩于[0,1]区间之中。

建议先试试看StandardScaler，效果不好换MinMaxScaler。

除了StandardScaler和MinMaxScaler之外，sklearn中也提供了各种其他缩放处理（中心化只需要一个pandas广播一下减去某个数就好了，因此sklearn不提供任何中心化功能）。比如，在希望压缩数据，却不影响数据的稀疏性时（不影响矩阵中取值为0的个数时），我们会使用MaxAbsScaler；在异常值多，噪声非常大时，我们可能会选用分位数来无量纲化，此时使用RobustScaler。更多详情请参考以下列表。

![image.png](https://pic4.zhimg.com/80/v2-beeec3ba7bfcb638bf9161bf09bd85b3_720w.jpg)

### 2.2 缺失值

机器学习和数据挖掘中所使用的数据，永远不可能是完美的。很多特征，对于分析和建模来说意义非凡，但对于实际收集数据的人却不是如此，因此数据挖掘之中，常常会有重要的字段缺失值很多，但又不能舍弃字段的情况。因此，数据预处理中非常重要的一项就是处理缺失值。

In [77]:
import pandas as pd
data = pd.read_csv("./datasets/Narrativedata.csv",index_col=0)

In [31]:
data.head()

Age     Sex Embarked Survived
0  22.0    male        S       No
1  38.0  female        C      Yes
2  26.0  female        S      Yes
3  35.0  female        S      Yes
4  35.0    male        S       No

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       714 non-null    float64
 1   Sex       891 non-null    object 
 2   Embarked  889 non-null    object 
 3   Survived  891 non-null    object 
dtypes: float64(1), object(3)
memory usage: 34.8+ KB


- **impute.SimpleImputer**

class sklearn.impute.SimpleImputer(missing_values=nan, strategy=’mean’, fill_value=None, verbose=0, copy=True)

在讲解随机森林的案例时，我们用这个类和随机森林回归填补了缺失值，对比了不同的缺失值填补方式对数据的影响。这个类是专门用来填补缺失值的。它包括四个重要参数：

![image.png](https://pic2.zhimg.com/80/v2-66c8d10439ac1bc58f06100843b478d5_720w.jpg)

In [36]:
data.loc[:,"Age"].values

array([22.  , 38.  , 26.  , 35.  , 35.  ,   nan, 54.  ,  2.  , 27.  ,
       14.  ,  4.  , 58.  , 20.  , 39.  , 14.  , 55.  ,  2.  ,   nan,
       31.  ,   nan, 35.  , 34.  , 15.  , 28.  ,  8.  , 38.  ,   nan,
       19.  ,   nan,   nan, 40.  ,   nan,   nan, 66.  , 28.  , 42.  ,
         nan, 21.  , 18.  , 14.  , 40.  , 27.  ,   nan,  3.  , 19.  ,
         nan,   nan,   nan,   nan, 18.  ,  7.  , 21.  , 49.  , 29.  ,
       65.  ,   nan, 21.  , 28.5 ,  5.  , 11.  , 22.  , 38.  , 45.  ,
        4.  ,   nan,   nan, 29.  , 19.  , 17.  , 26.  , 32.  , 16.  ,
       21.  , 26.  , 32.  , 25.  ,   nan,   nan,  0.83, 30.  , 22.  ,
       29.  ,   nan, 28.  , 17.  , 33.  , 16.  ,   nan, 23.  , 24.  ,
       29.  , 20.  , 46.  , 26.  , 59.  ,   nan, 71.  , 23.  , 34.  ,
       34.  , 28.  ,   nan, 21.  , 33.  , 37.  , 28.  , 21.  ,   nan,
       38.  ,   nan, 47.  , 14.5 , 22.  , 20.  , 17.  , 21.  , 70.5 ,
       29.  , 24.  ,  2.  , 21.  ,   nan, 32.5 , 32.5 , 54.  , 12.  ,
         nan, 24.  ,

In [39]:
Age = data.loc[:,"Age"].values.reshape(-1,1) #sklearn当中特征矩阵必须是二维

In [41]:
Age.shape

(891, 1)

In [42]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer()  #实例化，默认均值填补
imp_median = SimpleImputer(strategy="median") #用中位数填补
imp_0 = SimpleImputer(strategy="constant",fill_value=0) #用0填补

In [43]:
imp_mean = imp_mean.fit_transform(Age)   #fit_transform一步完成调取结果
imp_median = imp_median.fit_transform(Age)
imp_0 = imp_0.fit_transform(Age)

In [44]:
#在这里我们使用中位数填补Age
data.loc[:,"Age"] = imp_median

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       891 non-null    float64
 1   Sex       891 non-null    object 
 2   Embarked  889 non-null    object 
 3   Survived  891 non-null    object 
dtypes: float64(1), object(3)
memory usage: 34.8+ KB


In [46]:
#使用众数填补Embarked
Embarked = data.loc[:,"Embarked"].values.reshape(-1,1)
imp_mode = SimpleImputer(strategy = "most_frequent")
data.loc[:,"Embarked"] = imp_mode.fit_transform(Embarked)

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       891 non-null    float64
 1   Sex       891 non-null    object 
 2   Embarked  891 non-null    object 
 3   Survived  891 non-null    object 
dtypes: float64(1), object(3)
memory usage: 34.8+ KB


**用pandas进行处理：**

In [26]:
import pandas as pd
data_ = pd.read_csv("./datasets/Narrativedata.csv",index_col=0)
data_.head()

Age     Sex Embarked Survived
0  22.0    male        S       No
1  38.0  female        C      Yes
2  26.0  female        S      Yes
3  35.0  female        S      Yes
4  35.0    male        S       No

In [27]:
data_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       714 non-null    float64
 1   Sex       891 non-null    object 
 2   Embarked  889 non-null    object 
 3   Survived  891 non-null    object 
dtypes: float64(1), object(3)
memory usage: 34.8+ KB


In [28]:
data_.loc[:,"Age"] = data_.loc[:,"Age"].fillna(data_.loc[:,"Age"].median())
#.fillna 在DataFrame里面直接进行填补

In [29]:
data_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       891 non-null    float64
 1   Sex       891 non-null    object 
 2   Embarked  889 non-null    object 
 3   Survived  891 non-null    object 
dtypes: float64(1), object(3)
memory usage: 34.8+ KB


In [30]:
data_.dropna(axis=0,inplace=True)
#.dropna(axis=0)删除所有有缺失值的行，.dropna(axis=1)删除所有有缺失值的列
#参数inplace，为True表示在原数据集上进行修改，为False表示生成一个复制对象，不修改原数据，默认False

In [31]:
data_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       889 non-null    float64
 1   Sex       889 non-null    object 
 2   Embarked  889 non-null    object 
 3   Survived  889 non-null    object 
dtypes: float64(1), object(3)
memory usage: 34.7+ KB


### 2.3 处理分类型特征：编码与哑变量

在机器学习中，大多数算法，譬如逻辑回归，支持向量机SVM，k近邻算法等都只能够处理数值型数据，不能处理文字，在sklearn当中，除了专用来处理文字的算法，其他算法在fit的时候全部要求输入数组或矩阵，也不能够导入文字型数据（其实手写决策树和普斯贝叶斯可以处理文字，但是sklearn中规定必须导入数值型）。然而在现实中，许多标签和特征在数据收集完毕的时候，都不是以数字来表现的。比如说，学历的取值可以是["小学"，“初中”，“高中”，"大学"]，付费方式可能包含["支付宝"，“现金”，“微信”]等等。在这种情况下，为了让数据适应算法和库，我们必须将数据进行**编码**，即是说，**将文字型数据转换为数值型**。

- **preprocessing.LabelEncoder：标签专用，能够将分类转换为分类数值**

In [13]:
from sklearn.preprocessing import LabelEncoder

y = data.iloc[:,-1]    #要输入的是标签，不是特征矩阵，所以允许一维

In [14]:
y

0       No
1      Yes
2      Yes
3      Yes
4       No
      ... 
886     No
887    Yes
888     No
889    Yes
890     No
Name: Survived, Length: 891, dtype: object

In [11]:
le = LabelEncoder()      #实例化
le = le.fit(y)           #导入数据
label = le.transform(y)  #transform接口调取结果

In [12]:
label #查看获取的结果label

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [15]:
le.classes_   #属性.classes_查看标签中究竟有多少类别            

array(['No', 'Yes'], dtype=object)

In [ ]:
le.fit_transform(y)      #也可以直接fit_transform一步到位

le.inverse_transform(label) #使用inverse_transform可以逆转

In [16]:
data.iloc[:,-1] = label  #让标签等于我们运行出来的结果

In [17]:
data.head()

Age     Sex Embarked  Survived
0  22.0    male        S         0
1  38.0  female        C         1
2  26.0  female        S         1
3  35.0  female        S         1
4  35.0    male        S         0

In [ ]:
#如果不需要教学展示的话我会这么写：
from sklearn.preprocessing import LabelEncoder
data.iloc[:,-1] = LabelEncoder().fit_transform(data.iloc[:,-1])

- **preprocessing.OrdinalEncoder：特征专用，能够将分类特征转换为分类数值**

In [47]:
from sklearn.preprocessing import OrdinalEncoder

In [48]:
#接口categories_对应LabelEncoder的接口classes_，一模一样的功能
data2 = data_.copy()

In [49]:
data2.head()

Age     Sex Embarked Survived
0  22.0    male        S       No
1  38.0  female        C      Yes
2  26.0  female        S      Yes
3  35.0  female        S      Yes
4  35.0    male        S       No

In [50]:
from sklearn.preprocessing import LabelEncoder
data2.iloc[:,-1] = LabelEncoder().fit_transform(data2.iloc[:,-1])

In [51]:
OrdinalEncoder().fit(data2.iloc[:,1:-1]).categories_ 

[array(['female', 'male'], dtype=object), array(['C', 'Q', 'S'], dtype=object)]

In [52]:
data2.iloc[:,1:-1] = OrdinalEncoder().fit_transform(data2.iloc[:,1:-1])

In [55]:
data2.head()

Age  Sex  Embarked  Survived
0  22.0  1.0       2.0         0
1  38.0  0.0       0.0         1
2  26.0  0.0       2.0         1
3  35.0  0.0       2.0         1
4  35.0  1.0       2.0         0

- **preprocessing.OneHotEncoder：独热编码，创建哑变量**

我们刚才已经用OrdinalEncoder把分类变量Sex和Embarked都转换成数字对应的类别了。在舱门Embarked这一列中，我们使用[0,1,2]代表了三个不同的舱门，然而这种转换是正确的吗？<br>
<br>
我们来思考三种不同性质的分类数据：<br>
<br>
1） 舱门（S，C，Q）<br>
<br>
三种取值S，C，Q是相互独立的，彼此之间完全没有联系，表达的是S≠C≠Q的概念。这是名义变量。<br>
<br>
2） 学历（小学，初中，高中）<br>
<br>
三种取值不是完全独立的，我们可以明显看出，在性质上可以有高中>初中>小学这样的联系，学历有高低，但是学历取值之间却不是可以计算的，我们不能说小学 + 某个取值 = 初中。这是有序变量。<br>
<br>
3） 体重（>45kg，>90kg，>135kg）<br>
<br>
各个取值之间有联系，且是可以互相计算的，比如120kg - 45kg = 90kg，分类之间可以通过数学计算互相转换。这是有距变量。<br>

然而在对特征进行编码的时候，这三种分类数据都会被我们转换为[0,1,2]，这三个数字在算法看来，是连续且可以计算的，这三个数字相互不等，有大小，并且有着可以相加相乘的联系。所以算法会把舱门，学历这样的分类特征，都误会成是体重这样的分类特征。这是说，我们把分类转换成数字的时候，忽略了数字中自带的数学性质，所以给算法传达了一些不准确的信息，而这会影响我们的建模。<br>
<br>
类别OrdinalEncoder可以用来处理有序变量，但对于名义变量，我们只有使用哑变量的方式来处理，才能够尽量向算法传达最准确的信息： 

![image.png](https://pic1.zhimg.com/80/v2-99f178996727bba0fd5fcf36d7a67874_720w.jpg)

这样的变化，让算法能够彻底领悟，原来三个取值是没有可计算性质的，是“有你就没有我”的不等概念。在我们的数据中，性别和舱门，都是这样的名义变量。因此我们需要使用独热编码，将两个特征都转换为哑变量。

In [56]:
data.head()

Age     Sex Embarked  Survived
0  22.0    male        S         0
1  38.0  female        C         1
2  26.0  female        S         1
3  35.0  female        S         1
4  35.0    male        S         0

In [64]:
from sklearn.preprocessing import OneHotEncoder
X = data.iloc[:,1:-1]

In [65]:
X

Sex Embarked
0      male        S
1    female        C
2    female        S
3    female        S
4      male        S
..      ...      ...
886    male        S
887  female        S
888  female        S
889    male        C
890    male        Q

[891 rows x 2 columns]

In [66]:
enc = OneHotEncoder(categories='auto').fit(X)
result = enc.transform(X).toarray()
result

array([[0., 1., 0., 0., 1., 0.],
       [1., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 1., 0.],
       [0., 1., 1., 0., 0., 0.],
       [0., 1., 0., 1., 0., 0.]])

In [ ]:
#依然可以直接一步到位，但为了给大家展示模型属性，所以还是写成了三步
OneHotEncoder(categories='auto').fit_transform(X).toarray()

In [67]:
#依然可以还原
pd.DataFrame(enc.inverse_transform(result))

0  1
0      male  S
1    female  C
2    female  S
3    female  S
4      male  S
..      ... ..
886    male  S
887  female  S
888  female  S
889    male  C
890    male  Q

[891 rows x 2 columns]

In [68]:
enc.get_feature_names()

array(['x0_female', 'x0_male', 'x1_C', 'x1_Q', 'x1_S', 'x1_nan'],
      dtype=object)

In [69]:
result.shape

(891, 6)

In [70]:
#axis=1,表示跨行进行合并，也就是将两个表左右相连，如果是axis=0，就是将两个表上下相连
newdata = pd.concat([data,pd.DataFrame(result)],axis=1)

In [71]:
newdata.head()

Age     Sex Embarked  Survived    0    1    2    3    4    5
0  22.0    male        S         0  0.0  1.0  0.0  0.0  1.0  0.0
1  38.0  female        C         1  1.0  0.0  1.0  0.0  0.0  0.0
2  26.0  female        S         1  1.0  0.0  0.0  0.0  1.0  0.0
3  35.0  female        S         1  1.0  0.0  0.0  0.0  1.0  0.0
4  35.0    male        S         0  0.0  1.0  0.0  0.0  1.0  0.0

In [72]:
newdata.drop(["Sex","Embarked"],axis=1,inplace=True)

In [73]:
newdata.head()

Age  Survived    0    1    2    3    4    5
0  22.0         0  0.0  1.0  0.0  0.0  1.0  0.0
1  38.0         1  1.0  0.0  1.0  0.0  0.0  0.0
2  26.0         1  1.0  0.0  0.0  0.0  1.0  0.0
3  35.0         1  1.0  0.0  0.0  0.0  1.0  0.0
4  35.0         0  0.0  1.0  0.0  0.0  1.0  0.0

In [74]:
newdata.columns = ["Age","Survived","Female","Male","Embarked_C","Embarked_Q","Embarked_S","Embarked_NAN"]

In [75]:
newdata.head()

Age  Survived  Female  Male  Embarked_C  Embarked_Q  Embarked_S  \
0  22.0         0     0.0   1.0         0.0         0.0         1.0   
1  38.0         1     1.0   0.0         1.0         0.0         0.0   
2  26.0         1     1.0   0.0         0.0         0.0         1.0   
3  35.0         1     1.0   0.0         0.0         0.0         1.0   
4  35.0         0     0.0   1.0         0.0         0.0         1.0   

   Embarked_NAN  
0           0.0  
1           0.0  
2           0.0  
3           0.0  
4           0.0

特征可以做哑变量，标签也可以吗？可以，使用类sklearn.preprocessing.LabelBinarizer可以对做哑变量，许多算法都可以处理多标签问题（比如说决策树），但是这样的做法在现实中不常见，因此我们在这里就不赘述了。

![image.png](https://pic3.zhimg.com/80/v2-51212eeabf315c53666bf0f4c601b39a_720w.jpg)

### 2.4 处理连续型特征：二值化与分段

- **sklearn.preprocessing.Binarizer**

根据阈值将数据二值化（将特征值设置为0或1），用于处理连续型变量。大于阈值的值映射为1，而小于或等于阈值的值映射为0。默认阈值为0时，特征中所有的正值都映射到1。二值化是对文本计数数据的常见操作，分析人员可以决定仅考虑某种现象的存在与否。它还可以用作考虑布尔随机变量的估计器的预处理步骤（例如，使用贝叶斯设置中的伯努利分布建模）。

In [78]:
data.head()

Age     Sex Embarked Survived
0  22.0    male        S       No
1  38.0  female        C      Yes
2  26.0  female        S      Yes
3  35.0  female        S      Yes
4  35.0    male        S       No

In [79]:
#将年龄二值化

data_2 = data.copy()

In [80]:
from sklearn.preprocessing import Binarizer

In [81]:
X = data_2.iloc[:,0].values.reshape(-1,1)#类为特征专用，所以不能使用一维数组

In [82]:
X

array([[22.  ],
       [38.  ],
       [26.  ],
       [35.  ],
       [35.  ],
       [  nan],
       [54.  ],
       [ 2.  ],
       [27.  ],
       [14.  ],
       [ 4.  ],
       [58.  ],
       [20.  ],
       [39.  ],
       [14.  ],
       [55.  ],
       [ 2.  ],
       [  nan],
       [31.  ],
       [  nan],
       [35.  ],
       [34.  ],
       [15.  ],
       [28.  ],
       [ 8.  ],
       [38.  ],
       [  nan],
       [19.  ],
       [  nan],
       [  nan],
       [40.  ],
       [  nan],
       [  nan],
       [66.  ],
       [28.  ],
       [42.  ],
       [  nan],
       [21.  ],
       [18.  ],
       [14.  ],
       [40.  ],
       [27.  ],
       [  nan],
       [ 3.  ],
       [19.  ],
       [  nan],
       [  nan],
       [  nan],
       [  nan],
       [18.  ],
       [ 7.  ],
       [21.  ],
       [49.  ],
       [29.  ],
       [65.  ],
       [  nan],
       [21.  ],
       [28.5 ],
       [ 5.  ],
       [11.  ],
       [22.  ],
       [38.  ],
       [

In [84]:
data_2.loc[:,"Age"] = data_2.loc[:,"Age"].fillna(data_2.loc[:,"Age"].median())
#threshold 38以上的划为1，以下的划为0
transformer = Binarizer(threshold=38).fit_transform(X) 

In [85]:
transformer

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [89]:
data_2.iloc[:,0] = transformer

In [90]:
data_2.head()

Age     Sex Embarked Survived
0  0.0    male        S       No
1  0.0  female        C      Yes
2  0.0  female        S      Yes
3  0.0  female        S      Yes
4  0.0    male        S       No

- **preprocessing.KBinsDiscretizer**

这是将连续型变量划分为分类变量的类，能够将连续型变量排序后按顺序分箱后编码。总共包含三个重要参数：

![image.png](https://pic4.zhimg.com/80/v2-12ed73205151ff8f4ae57ce0ba5aac03_720w.jpg)

In [96]:
from sklearn.preprocessing import KBinsDiscretizer

In [97]:
data_3 = data_.copy()

In [98]:
data_3.head()

Age     Sex Embarked Survived
0  22.0    male        S       No
1  38.0  female        C      Yes
2  26.0  female        S      Yes
3  35.0  female        S      Yes
4  35.0    male        S       No

In [99]:
X = data_3.iloc[:,0].values.reshape(-1,1)

In [100]:
est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
est.fit_transform(X)

array([[0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [2.],
       [0.],
       [1.],
       [0.],
       [0.],
       [2.],
       [0.],
       [1.],
       [0.],
       [2.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [2.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [2.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],

In [101]:
#查看转换后分的箱：变成了一列中的三箱
set(est.fit_transform(X).ravel())

{0.0, 1.0, 2.0}

In [102]:
est = KBinsDiscretizer(n_bins=3, encode='onehot', strategy='uniform')
#查看转换后分的箱：变成了哑变量
est.fit_transform(X).toarray()

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])